<a href="https://colab.research.google.com/github/abissankar/ChatBot-KEC/blob/main/CHATBOT_KONGU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q torch datasets accelerate peft bitsandbytes trl
!pip install langchain playwright html2text sentence_transformers faiss-gpu gradio
!pip install langchain-community
!playwright install chromium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
# Import required modules
import transformers
from transformers import AutoTokenizer, BitsAndBytesConfig, AutoModelForCausalLM
import torch
from langchain.memory import ConversationBufferMemory
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import AsyncChromiumLoader
from langchain.document_transformers import Html2TextTransformer
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
import nest_asyncio
import gradio as gr

In [3]:
# Set up asyncio
nest_asyncio.apply()

In [4]:
# Set model and token details
model_name = "ministral/Ministral-3b-instruct"






In [5]:
from huggingface_hub import login
login()   # will ask for your new HF token


In [11]:
# Initialize model configuration
model_config = transformers.AutoConfig.from_pretrained(
    model_name, trust_remote_code=True
)


In [12]:
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True
)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [13]:
# BitsAndBytes configuration
use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

In [14]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

In [15]:
# Load the model with 4-bit precision
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/2.96G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/698M [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/2.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [71]:
# Install Hugging Face Transformers if not already
!pip install transformers accelerate bitsandbytes -q

#  Imports
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

In [110]:
# Build pipeline
text_generation_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    do_sample=False,         # deterministic output
    repetition_penalty=1.1,
    return_full_text=False,  # ✅ only the model's answer
    max_new_tokens=300,
    pad_token_id=tokenizer.eos_token_id
)

Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [111]:
# Setup LangChain memory for conversation history
memory = ConversationBufferMemory(memory_key="chat_history", input_key="human_input", return_messages=True)

In [112]:
# Define the prompt template
prompt_template = """
### [INST]
Instruction: Answer the question based on your
Kongu Engineering College knowledge. Here is context to help:

{context}

### QUESTION:
{question}

[/INST]
"""

In [113]:
# Initialize HuggingFacePipeline with the model
mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

In [114]:
# Create a prompt using the template
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

In [115]:
# Create the LLMChain
llm_chain = LLMChain(llm=mistral_llm, prompt=prompt, memory=memory)

In [116]:
# Setup embedding and retriever (for RAG)
# Define the articles to scrape
articles = [
    "https://en.wikipedia.org/wiki/Kongu_Engineering_College",
    "https://www.kongu.edu/aboutus.html"

]

In [117]:
# Scrape the articles for context
!pip install playwright
!playwright install

loader = AsyncChromiumLoader(articles)
docs = loader.load()

Playwright Host validation warning: 
╔══════════════════════════════════════════════════════╗
║ Host system is missing dependencies to run browsers. ║
║ Missing libraries:                                   ║
║     libwoff2dec.so.1.0.2                             ║
║     libgstgl-1.0.so.0                                ║
║     libgstcodecparsers-1.0.so.0                      ║
║     libavif.so.13                                    ║
║     libharfbuzz-icu.so.0                             ║
║     libenchant-2.so.2                                ║
║     libsecret-1.so.0                                 ║
║     libhyphen.so.0                                   ║
║     libmanette-0.2.so.0                              ║
╚══════════════════════════════════════════════════════╝
    at validateDependenciesLinux (/usr/local/lib/python3.11/dist-packages/playwright/driver/package/lib/server/registry/dependencies.js:269:9)
    at process.processTicksAndRejections (node:internal/process/task_queues:105

In [118]:
# Convert HTML to plain text
!pip install html2text

html2text = Html2TextTransformer()
docs_transformed = html2text.transform_documents(docs)

In [119]:
!pip install faiss-cpu


In [120]:
# Chunk the text for indexing
text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=0)
chunked_documents = text_splitter.split_documents(docs_transformed)
# Load chunked documents into FAISS index
db = FAISS.from_documents(chunked_documents, HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'))

In [121]:
# Convert FAISS into a retriever
retriever = db.as_retriever(search_type="similarity", search_kwargs={'k': 4})

In [127]:
# Improved chatbot interaction function
def chatbot_interaction(context, question):
    # Retrieve context if not provided
    if not context.strip():
        docs = retriever.get_relevant_documents(question)
        context = "\n".join([doc.page_content for doc in docs])

    # Debug print to see what context is retrieved
    print("DEBUG: Retrieved context:\n", context)

    # More precise prompt to minimize hallucination
    query = f"""
Use ONLY the information in the context below to answer the question.
If the answer is not contained in the context, respond with "I don't know."

Context:
{context}

Question:
{question}
"""
    response = text_generation_pipeline(query)[0]["generated_text"]
    return response.strip()


In [124]:
# Create Gradio app
iface = gr.Interface(
    fn=chatbot_interaction,
    inputs=[
        gr.Textbox(lines=5, label="Context (optional)", placeholder="Paste context here or leave blank for auto-fetch"),
        gr.Textbox(lines=2, label="Question", placeholder="Enter your question here")
    ],
    outputs=gr.Textbox(label="Response"),
    title="Kongu Engineering College Chatbot",
    description="Ask questions about Kongu Engineering College. Provide context or let the chatbot retrieve information!"
)

In [125]:
# Launch Gradio app
iface.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b908cf95c0711d656f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
